In [35]:
#| default_exp datasets

In [ ]:
#|export
import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl,numpy as np,matplotlib.pyplot as plt
from pathlib import Path
from operator import itemgetter

from torch import tensor,nn,optim
from torch.utils.data import DataLoader,default_collate
import torch.nn.functional as F

from datasets import load_dataset,load_dataset_builder

In [19]:
import torchvision.transforms.functional as TF
from fastcore.test import test_close

torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
torch.manual_seed(1)
mpl.rcParams['image.cmap'] = 'gray'

import logging
logging.disable(logging.WARNING)

## Hugging Face Datasets

In [20]:
name = "fashion_mnist"
ds_builder = load_dataset_builder(name)
print(ds_builder.info.description)

Fashion-MNIST is a dataset of Zalando's article images—consisting of a training set of
60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image,
associated with a label from 10 classes. We intend Fashion-MNIST to serve as a direct drop-in
replacement for the original MNIST dataset for benchmarking machine learning algorithms.
It shares the same image size and structure of training and testing splits.



In [21]:
ds_builder.info.features

{'image': Image(decode=True, id=None),
 'label': ClassLabel(num_classes=10, names=['T - shirt / top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'], id=None)}

In [22]:
ds_builder.info.splits

{'train': SplitInfo(name='train', num_bytes=31296607, num_examples=60000, dataset_name='fashion_mnist'),
 'test': SplitInfo(name='test', num_bytes=5233810, num_examples=10000, dataset_name='fashion_mnist')}

In [23]:
dsd = load_dataset(name)
dsd

  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 60000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 10000
    })
})

In [24]:
train,test = dsd['train'],dsd['test']
train[0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
 'label': 9}

In [25]:
x,y = 'image','label'
img = train[0][x]
img

In [26]:
xb = train[:5][x]
yb = train[:5][y]
yb

[9, 0, 0, 3, 0]

In [27]:
featy = train.features[y]
featy

ClassLabel(num_classes=10, names=['T - shirt / top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'], id=None)

In [28]:
featy.int2str(yb)

['Ankle boot',
 'T - shirt / top',
 'T - shirt / top',
 'Dress',
 'T - shirt / top']

In [29]:
train['label'][:5]

[9, 0, 0, 3, 0]

In [30]:
def collate_fn(b):
    return {x:torch.stack([TF.to_tensor(o[x]) for o in b]),
            y:tensor([o[y] for o in b])}

In [31]:
dl = DataLoader(train, collate_fn=collate_fn, batch_size=4)
b = next(iter(dl))
b[x].shape,b[y]

(torch.Size([4, 1, 28, 28]), tensor([9, 0, 0, 3]))

In [32]:
def transforms(b):
    b[x] = [TF.to_tensor(o) for o in b[x]]
    return b

In [33]:
tds = train.with_transform(transforms)
dl = DataLoader(tds, batch_size=4)
b = next(iter(dl))
b[x].shape,b[y]

(torch.Size([4, 1, 28, 28]), tensor([9, 0, 0, 3]))

In [34]:
#|export
def inplace(f):
    def _f(b):
        f(b)
        return b
    return _f

In [35]:
@inplace
def transformi(b): b[x] = [torch.flatten(TF.to_tensor(o)) for o in b[x]]

In [36]:
tds = train.with_transform(transformi)
r = tds[0]
r[x].shape,r[y]

(torch.Size([784]), 9)

In [38]:
d = dict(a=1,b=2,c=3)
itemgetter('a','c')(d)

(1, 3)

In [39]:
#|export
def collate_dict(ds):
    get = itemgetter(*ds.features)
    def _f(b): return get(default_collate(b))
    return _f

In [40]:
dl = DataLoader(tds, batch_size=4, collate_fn=collate_dict(tds))
xb,xy = next(iter(dl))
xb.shape,yb

(torch.Size([4, 784]), [9, 0, 0, 3, 0])

## Export -

In [37]:
import nbdev; nbdev.nbdev_export()